# Binarized Neural Networks



Inês Jesus, 2073570

Auriane Mahfouz, 2072042

## Importing Libraries

In [251]:
! pip install python-sat

In [252]:
import itertools
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from pysat.formula import WCNF
import time
from pysat.examples.rc2 import RC2
from pysat.examples.fm import FM
from sklearn.metrics import accuracy_score

In [253]:
NUM_INPUT = 5

## Generating Dataset

In [254]:
inputs = list(itertools.product([-1,1], repeat=NUM_INPUT))

In [255]:
for x in inputs:
  print(x)

(-1, -1, -1, -1, -1)
(-1, -1, -1, -1, 1)
(-1, -1, -1, 1, -1)
(-1, -1, -1, 1, 1)
(-1, -1, 1, -1, -1)
(-1, -1, 1, -1, 1)
(-1, -1, 1, 1, -1)
(-1, -1, 1, 1, 1)
(-1, 1, -1, -1, -1)
(-1, 1, -1, -1, 1)
(-1, 1, -1, 1, -1)
(-1, 1, -1, 1, 1)
(-1, 1, 1, -1, -1)
(-1, 1, 1, -1, 1)
(-1, 1, 1, 1, -1)
(-1, 1, 1, 1, 1)
(1, -1, -1, -1, -1)
(1, -1, -1, -1, 1)
(1, -1, -1, 1, -1)
(1, -1, -1, 1, 1)
(1, -1, 1, -1, -1)
(1, -1, 1, -1, 1)
(1, -1, 1, 1, -1)
(1, -1, 1, 1, 1)
(1, 1, -1, -1, -1)
(1, 1, -1, -1, 1)
(1, 1, -1, 1, -1)
(1, 1, -1, 1, 1)
(1, 1, 1, -1, -1)
(1, 1, 1, -1, 1)
(1, 1, 1, 1, -1)
(1, 1, 1, 1, 1)


Defining some node functions:

In [256]:
# Node function 1 to return majority of the input nodes
def majority_function(x):
    return 1 if sum(x) > 0 else -1

In [257]:
# Node function 2 returns parity of positive nodes
def parity_function(x):
    count_ones = sum(1 for xi in x if xi == 1)
    return 1 if count_ones % 2 == 0 else -1

In [258]:
# Node function 3 returns if the first n or last n nodes are all positive
def first_or_last_n(x,n):
    first_n = sum([x[i] == 1 for i in range(n)]) == n
    last_n = sum([x[i] == 1 for i in range(-n,0)]) == n
    first_or_last = first_n or last_n
    return 1 if first_or_last else -1

In [259]:
# Node function 4 returns negation of input node i, i=1,...,5
def negation_input(x,i):
    return -x[i-1]

In [260]:
# Node function 5 returns XNOR operator on inputs i and j, i,j = 1,...,5
def xnor(x,i,j):
    xnor_result = (x[i-1] == -1 and x[j-1] == -1) or (x[i-1] == 1 and x[j-1] == 1)
    return 1 if xnor_result else -1

Assigning labels based on the five node functions:

In [261]:
# # function f1
# targets = [(x[0] and x[1],
#             x[2] and x[3],
#             x[1] and x[4],
#             x[0] and x[3],
#             x[2] and x[4])
#            for x in inputs]

In [262]:
# function f2
targets = [(x[0] or (x[1] and x[2]),
            -x[1],
            ((x[0] or x[1]) and (x[2] or x[3])) or (x[4] and (-x[0] and -x[2])),
            (x[0] and x[1] and x[2]) or (x[3] and x[4]),
            -x[1] and -x[3] and -x[4])
            for x in inputs]

In [263]:
# # function f3
# targets = [[majority_function(x),
#             parity_function(x),
#             first_or_last_n(x,2),
#             negation_input(x,3),
#             xnor(x,2,3)]
#             for x in inputs]

In [264]:
targets

[(-1, 1, -1, -1, 1),
 (-1, 1, -1, -1, -1),
 (-1, 1, -1, -1, 1),
 (-1, 1, -1, -1, -1),
 (-1, 1, 1, 1, 1),
 (-1, 1, 1, 1, -1),
 (-1, 1, 1, 1, 1),
 (-1, 1, 1, 1, -1),
 (-1, -1, -1, -1, 1),
 (-1, -1, -1, -1, -1),
 (-1, -1, -1, -1, 1),
 (-1, -1, -1, -1, -1),
 (-1, -1, 1, 1, 1),
 (-1, -1, 1, 1, -1),
 (-1, -1, 1, 1, 1),
 (-1, -1, 1, 1, -1),
 (1, 1, -1, -1, 1),
 (1, 1, -1, -1, -1),
 (1, 1, -1, -1, 1),
 (1, 1, -1, -1, -1),
 (1, 1, 1, 1, 1),
 (1, 1, 1, 1, -1),
 (1, 1, 1, 1, 1),
 (1, 1, 1, 1, -1),
 (1, -1, -1, -1, 1),
 (1, -1, -1, -1, -1),
 (1, -1, -1, -1, 1),
 (1, -1, -1, -1, -1),
 (1, -1, 1, 1, 1),
 (1, -1, 1, 1, -1),
 (1, -1, 1, 1, 1),
 (1, -1, 1, 1, -1)]

In [265]:
X_train, X_test, y_train, y_test = train_test_split(inputs,
                                                    targets,
                                                    test_size=0.25,
                                                    random_state=42)

print(len(X_train))
print(len(X_test))

24
8


Check balance per function, because it importance to have a nice balance at least in the training dataset, so that the model doesn't learn to predict all 1's are all -1's for a certain function

In [266]:
NUM_OUTPUT = len(targets[0])

In [267]:
NUM_OUTPUT

5

In [268]:
for i, dataset in enumerate([targets, y_train, y_test]):
  dict = {0: "Targets", 1: "Training set", 2: "Test set"}
  print(dict[i])
  dataset_data = []

  for function in range(NUM_OUTPUT):
    pos_input = 0
    for y in dataset:
      if y[function]==1: pos_input+=1
    positive_ratio = np.round(pos_input/len(dataset),2)
    negative_ratio = np.round((len(dataset)-pos_input)/len(dataset),2)

    dataset_data.append([positive_ratio, negative_ratio])

  df = pd.DataFrame(dataset_data, columns=['Positive', 'Negative'], index= list(range(1,NUM_OUTPUT+1)))

  print(df)
  print()

Targets
   Positive  Negative
1       0.5       0.5
2       0.5       0.5
3       0.5       0.5
4       0.5       0.5
5       0.5       0.5

Training set
   Positive  Negative
1      0.46      0.54
2      0.62      0.38
3      0.54      0.46
4      0.54      0.46
5      0.54      0.46

Test set
   Positive  Negative
1      0.62      0.38
2      0.12      0.88
3      0.38      0.62
4      0.38      0.62
5      0.38      0.62



## Variable indexing

In [269]:
layers = [5, 7, NUM_OUTPUT]

In [270]:
def neuron_index(t, n_layer, i):
    """
    t: training sample index
    n_layer: layer index (input is layer 1)
    i: neuron index
    """
    cum_sum = np.sum(layers[0:n_layer-1]) * len(X_train)
    return int(cum_sum + t * layers[n_layer-1] + i + 1)

def weight_index(n_layer, i, j):
    """
    n_layer: layer index, starts in 1
    i: previous neuron index
    j: next neuron index
    """
    n_neurons = np.sum(layers) * len(X_train)
    cum_weight_sum = sum(x * y for x, y in zip(layers[0:n_layer-1], layers[1:n_layer]))
    return int(n_neurons + cum_weight_sum + i * layers[n_layer] + j + 1)

In [271]:
# first input
neuron_index(0,1,0)

1

In [272]:
# last input
neuron_index(len(X_train)-1,1,layers[0]-1)

120

In [273]:
# first hidden neuron (or output in case there is no hidden layer)
neuron_index(0,2,0)

121

In [274]:
# last hidden neuron (or output in case there is no hidden layer)
neuron_index(len(X_train)-1,2,layers[-2]-1)

288

In [275]:
# first output
neuron_index(0,len(layers),0)

289

In [276]:
# last output
neuron_index(len(X_train)-1,len(layers),NUM_OUTPUT-1)

408

In [277]:
# weight between first neuron of input layer and first neuron of first hidden layer
weight_index(1,0,0)

409

In [278]:
# weight between last neuron of last hidden layer and last neuron of output layer
weight_index(len(layers)-1,layers[-2]-1,layers[-1]-1)

478

## BNN encoding

In [279]:
wcnf = WCNF()

In [280]:
print("For first sample", X_train[0])
for subset in itertools.combinations(range(layers[0]), layers[0] // 2 + 1):
  neuron_next_x = neuron_index(0,2,0)
  print("Chosen indexes", subset)
  print("Respective neurons", list(X_train[0][idx] for idx in subset))
  w_pos_clause = [weight_index(1,neuron,0) * -signal for neuron, signal in zip(subset,list(X_train[0][idx] for idx in subset))]
  pos_clause = [int(a) for a in w_pos_clause + [neuron_next_x]]
  print("To make next neuron positive", pos_clause)
  print("And to make it negative", [-a for a in pos_clause])
  print("If neither all weights are of same sign or of different sign as input, this neuron combination doesn't influence the next neuron")
  break

For first sample (-1, 1, 1, -1, -1)
Chosen indexes (0, 1, 2)
Respective neurons [-1, 1, 1]
To make next neuron positive [409, -416, -423, 121]
And to make it negative [-409, 416, 423, -121]
If neither all weights are of same sign or of different sign as input, this neuron combination doesn't influence the next neuron


In [281]:
def add_first_layer(wcnf_formula):
  n_next_neurons = layers[1]
  for t in range(len(X_train)):
    for j in range(n_next_neurons):
      neuron_next_x = neuron_index(t,2,j)
      input_selection = itertools.combinations(range(layers[0]), layers[0] // 2 + 1)
      for subset in input_selection:
        input_values = list(X_train[t][idx] for idx in subset)

        # make next neuron positive
        w_pos_clause = [weight_index(1,neuron,j) * -signal for neuron, signal in zip(subset,input_values)]
        pos_clause = [int(a) for a in w_pos_clause+[neuron_next_x]]
        wcnf_formula.append(pos_clause)

        # make next neuron negative
        neg_clause = [-a for a in pos_clause]
        wcnf_formula.append(neg_clause)

  return wcnf_formula

In [282]:
def add_layer(layer_n,wcnf_formula):
  n_prev_neurons = layers[layer_n-1]
  n_next_neurons = layers[layer_n]
  for t in range(len(X_train)):
    # add hard clauses for connection between neurons
    for j in range(n_next_neurons):
      neuron_next_x = neuron_index(t,layer_n+1,j)
      input_selection = itertools.combinations(range(n_prev_neurons), n_prev_neurons // 2 + 1)
      for subset in input_selection:
        input_values = itertools.product([-1, 1], repeat= n_prev_neurons // 2 + 1)
        for comb in input_values:
          x_clause = [neuron_index(t,layer_n,subset[neuron])*signal for neuron,signal in enumerate(comb)]

          w_pos_clause = [weight_index(layer_n,subset[neuron],j)*signal for neuron,signal in enumerate(comb)]
          pos_clause = [int(a) for a in x_clause+w_pos_clause+[neuron_next_x]]
          wcnf_formula.append(pos_clause)

          w_neg_clause = [- weight for weight in w_pos_clause]
          neg_clause = [int(a) for a in x_clause+w_neg_clause+[-1*neuron_next_x]]
          wcnf_formula.append(neg_clause)
  return wcnf_formula

In [283]:
def add_output_layer(wcnf_formula):
  for t in range(len(y_train)):
    for j in range(NUM_OUTPUT):
      y_clause = [neuron_index(t,len(layers),j)*y_train[t][j]]
      wcnf_formula.append(y_clause, weight = 1)
  return wcnf_formula

In [284]:
start = time.time()
for i,layer in enumerate(layers):
  if i == 0:
    wcnf = add_first_layer(wcnf)
  elif i < len(layers)-1:
    # add hard clauses of connections between neurons
    wcnf = add_layer(i+1,wcnf)
  else:
    # add soft clauses of output
    wcnf = add_output_layer(wcnf)
layers_clauses_time = time.time()-start

In [285]:
wcnf.hard[:5]

[[409, -416, -423, 121],
 [-409, 416, 423, -121],
 [409, -416, 430, 121],
 [-409, 416, -430, -121],
 [409, -416, 437, 121]]

In [286]:
wcnf.hard[-5:]

[[285, 286, -287, 288, -463, -468, 473, -478, -408],
 [285, 286, 287, -288, 463, 468, 473, -478, 408],
 [285, 286, 287, -288, -463, -468, -473, 478, -408],
 [285, 286, 287, 288, 463, 468, 473, 478, 408],
 [285, 286, 287, 288, -463, -468, -473, -478, -408]]

In [287]:
wcnf.soft[-5:]

[[-404], [405], [406], [407], [408]]

In [288]:
len(wcnf.hard)

137760

In [289]:
len(wcnf.soft)

120

In [290]:
wcnf_copy = wcnf.copy()

In [291]:
solver = RC2(wcnf_copy)
# solver = FM(wcnf_copy)

In [292]:
start = time.time()
model = solver.compute()
model_compute_time = time.time()-start

In [293]:
solver.cost

0

In [294]:
print(solver.model)

[-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25, -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38, -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51, -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64, -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77, -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90, -91, -92, -93, -94, -95, -96, -97, -98, -99, -100, -101, -102, -103, -104, -105, -106, -107, -108, -109, -110, -111, -112, -113, -114, -115, -116, -117, -118, -119, -120, 121, 122, -123, -124, -125, -126, 127, 128, 129, -130, 131, -132, 133, 134, -135, 136, -137, 138, -139, -140, 141, -142, 143, -144, 145, -146, -147, -148, -149, 150, -151, 152, 153, 154, 155, -156, -157, -158, 159, -160, -161, -162, 163, -164, 165, 166, -167, 168, -169, -170, 171, 172, -173, 174, 175, 176, -177, -178, -179, 180, -181, 182, -183, 184, -185, -186, 187, -188, 189, 190, 1

In [295]:
len(model)

478

In [296]:
for i in range(1,len(layers)):
  w_i = np.sign(solver.model[weight_index(i,0,0)-1:weight_index(i,layers[i-1]-1,layers[i]-1)]).reshape(layers[i-1],layers[i])
  print("W"+str(i)+":")
  print(w_i)
  print()
  if i == 1:
    train_h_i = np.sign(np.matmul(X_train,w_i))
    test_h_i = np.sign(np.matmul(X_test,w_i))
  else:
    train_h_i = np.sign(np.matmul(train_h_i,w_i))
    test_h_i = np.sign(np.matmul(test_h_i,w_i))

W1:
[[-1  1  1 -1  1 -1 -1]
 [ 1 -1  1 -1 -1 -1 -1]
 [-1  1 -1 -1  1 -1  1]
 [ 1 -1  1 -1  1  1  1]
 [-1 -1  1  1  1  1 -1]]

W2:
[[-1 -1 -1 -1  1]
 [ 1  1 -1 -1  1]
 [ 1 -1 -1 -1 -1]
 [-1  1 -1 -1 -1]
 [ 1  1  1  1 -1]
 [-1  1 -1 -1 -1]
 [-1  1  1  1  1]]



In [297]:
pred_train = train_h_i
pred_test = test_h_i

## Evaluation

In [298]:
train_acc = accuracy_score(list(itertools.chain(*np.sign(np.array(y_train)))), list(itertools.chain(*pred_train)))
test_acc = accuracy_score(list(itertools.chain(*np.sign(np.array(y_test)))), list(itertools.chain(*pred_test)))

print(f"Training acc: {train_acc}")
print(f"Test acc: {test_acc}")

Training acc: 1.0
Test acc: 1.0


In [299]:
y_test == pred_test

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

#### Accuracy per function

In [300]:
(y_test == pred_test).sum(axis=0)

array([8, 8, 8, 8, 8])

In [301]:
#on test
test_acc_per_func = (y_test == pred_test).sum(axis=0)/np.array(y_test).shape[0]
print(test_acc_per_func)

[1. 1. 1. 1. 1.]


In [302]:
train_acc_per_func = (y_train == pred_train).sum(axis=0)/np.array(y_train).shape[0]
print(train_acc_per_func)

[1. 1. 1. 1. 1.]


To remember the functions:

```
targets = [(majority_function(x),
            parity_function(x),
            first_or_last_n(x,2),
            negation_input(x,3),
            xnor(x,2,3)) for x in inputs]
```



#### Accuracy per sample

In [303]:
np.round(np.sum(np.array(y_test) == np.array(pred_test), axis=1)/np.array(y_test).shape[1],2)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [304]:
np.round(np.sum(np.array(y_train) == np.array(pred_train), axis=1)/np.array(y_train).shape[1],2)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1.])

## Summary

In [305]:
# Formalization
print("Layers:", layers)
print("Train size:", len(X_train))
print()

# Clauses
print("Number of hard clauses:", len(wcnf.hard))
print("Number of soft clauses:", len(wcnf.soft))
print("Time to add clauses: %dmin %02dsec" % (layers_clauses_time / 60, layers_clauses_time % 60))
print()

# Model and solver
print("Model computing time:  %dmin %02dsec" % (model_compute_time / 60, model_compute_time % 60))
print("Solver cost:", solver.cost)
print("Size of model:", len(model))
print()

# Evaluation
print("Train accuracy:", np.round(train_acc,2))
print("Train accuracy per function:", np.round(train_acc_per_func,2))
print("Test accuracy:", np.round(test_acc,2))
print("Test accuracy per function:", np.round(test_acc_per_func,2))

Layers: [5, 7, 5]
Train size: 24

Number of hard clauses: 137760
Number of soft clauses: 120
Time to add clauses: 0min 11sec

Model computing time:  0min 01sec
Solver cost: 0
Size of model: 478

Train accuracy: 1.0
Train accuracy per function: [1. 1. 1. 1. 1.]
Test accuracy: 1.0
Test accuracy per function: [1. 1. 1. 1. 1.]
